In [15]:
! pip install somajo

     |████████████████████████████████| 81kB 1.5MB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/6f/4e/1b178c38c9a1a184288f72065a65ca01f3154df43c6ad898624149b8b4e0/regex-2019.06.08.tar.gz
  Created wheel for somajo: filename=SoMaJo-1.10.3-cp37-none-any.whl size=78265 sha256=691d100218ba4512a8287fc085aff5372d3053c1e6c483162596e7c8b0d4b090
  Stored in directory: /home/filter/.cache/pip/wheels/50/58/d8/a18ca802e6676dc202bc05988712ce4c4878d6e06533e76dfc
  Created wheel for regex: filename=regex-2019.6.8-cp37-cp37m-linux_x86_64.whl size=610596 sha256=23bfeffe045360210acdb82b5a0c64945e99a88fbe8a35ec45f6eed8ce78d78f
  Stored in directory: /home/filter/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built somajo regex


In [10]:
import datetime
import json
import pickle
from pathlib import Path

# import dateparser
import pandas as pd
from cleantext import clean
# from german_lemmatizer import lemmatize
from joblib import Parallel, delayed
from somajo import SentenceSplitter, Tokenizer
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

In [3]:
import stanfordnlp
# stanfordnlp.download('de')   # This downloads the English models for the neural pipeline

In [4]:
bs = 1000

In [11]:
nlp = stanfordnlp.Pipeline(pos_batch_size=bs * 10, mwt_batch_size=bs,tokenize_pretokenized=True,tokenize_batch_size=bs, lemma_batch_size=bs, models_dir='/mnt/data/stanfordnlp_resources', lang='de', processors='tokenize,lemma') # This sets up a default neural pipeline in English

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/mnt/data/stanfordnlp_resources/de_gsd_models/de_gsd_tokenizer.pt', 'pretokenized': True, 'batch_size': 1000, 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/mnt/data/stanfordnlp_resources/de_gsd_models/de_gsd_lemmatizer.pt', 'batch_size': 1000, 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
Done loading processors!
---


In [12]:
STOP_WORDS = set(
    """
á a ab aber ach acht achte achten achter achtes ag alle allein allem allen
aller allerdings alles allgemeinen als also am an andere anderen anderem andern
anders auch auf aus ausser außer ausserdem außerdem
bald bei beide beiden beim beispiel bekannt bereits besonders besser besten bin
bis bisher bist
da dabei dadurch dafür dagegen daher dahin dahinter damals damit danach daneben
dank dann daran darauf daraus darf darfst darin darüber darum darunter das
dasein daselbst dass daß dasselbe davon davor dazu dazwischen dein deine deinem
deiner dem dementsprechend demgegenüber demgemäss demgemäß demselben demzufolge
den denen denn denselben der deren derjenige derjenigen dermassen dermaßen
derselbe derselben des deshalb desselben dessen deswegen dich die diejenige
diejenigen dies diese dieselbe dieselben diesem diesen dieser dieses dir doch
dort drei drin dritte dritten dritter drittes du durch durchaus dürfen dürft
durfte durften
eben ebenso ehrlich eigen eigene eigenen eigener eigenes ein einander eine
einem einen einer eines einigeeinigen einiger einiges einmal einmaleins elf en
ende endlich entweder er erst erste ersten erster erstes es etwa etwas euch
früher fünf fünfte fünften fünfter fünftes für
gab ganz ganze ganzen ganzer ganzes gar gedurft gegen gegenüber gehabt gehen
geht gekannt gekonnt gemacht gemocht gemusst genug gerade gern gesagt geschweige
gewesen gewollt geworden gibt ging gleich gott gross groß grosse große grossen
großen grosser großer grosses großes gut gute guter gutes
habe haben habt hast hat hatte hätte hatten hätten heisst heißt her heute hier
hin hinter hoch
ich ihm ihn ihnen ihr ihre ihrem ihren ihrer ihres im immer in indem
infolgedessen ins irgend ist
ja jahr jahre jahren je jede jedem jeden jeder jedermann jedermanns jedoch
jemand jemandem jemanden jene jenem jenen jener jenes jetzt
kam kann kannst kaum kein keine keinem keinen keiner kleine kleinen kleiner
kleines kommen kommt können könnt konnte könnte konnten kurz
lang lange leicht leider lieber los
machen macht machte mag magst man manche manchem manchen mancher manches mehr
mein meine meinem meinen meiner meines mensch menschen mich mir mit mittel
mochte möchte mochten mögen möglich mögt morgen muss muß müssen musst müsst
musste mussten
na nach nachdem nahm natürlich neben nein neue neuen neun neunte neunten neunter
neuntes nicht nichts nie niemand niemandem niemanden noch nun nur
ob oben oder offen oft ohne
recht rechte rechten rechter rechtes richtig rund
sagt sagte sah satt schlecht schon sechs sechste sechsten sechster sechstes
sehr sei seid seien sein seine seinem seinen seiner seines seit seitdem selbst
selbst sich sie sieben siebente siebenten siebenter siebentes siebte siebten
siebter siebtes sind so solang solche solchem solchen solcher solches soll
sollen sollte sollten sondern sonst sowie später statt
tag tage tagen tat teil tel trotzdem tun
über überhaupt übrigens uhr um und uns unser unsere unserer unter
vergangene vergangenen viel viele vielem vielen vielleicht vier vierte vierten
vierter viertes vom von vor
wahr während währenddem währenddessen wann war wäre waren wart warum was wegen
weil weit weiter weitere weiteren weiteres welche welchem welchen welcher
welches wem wen wenig wenige weniger weniges wenigstens wenn wer werde werden
werdet wessen wie wieder will willst wir wird wirklich wirst wo wohl wollen
wollt wollte wollten worden wurde würde wurden würden
zehn zehnte zehnten zehnter zehntes zeit zu zuerst zugleich zum zunächst zur
zurück zusammen zwanzig zwar zwei zweite zweiten zweiter zweites zwischen
""".split()
)

In [13]:
def chunks(l, n):

    """
    Yield successive n-sized chunks from l.
    """
    for i in range(0, len(l), n):
        yield l[i : i + n]


In [7]:
df = pd.read_pickle('parsed_data.pkl')

In [16]:
def run(c):
    final_text = ''
    tokenizer = Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
    sentence_splitter = SentenceSplitter(is_tuple=False)
    for text in c:
        text = clean(text, lang='de', lower=False)
        tokens = tokenizer.tokenize_paragraph(text)
        final_text += ' '.join(tokens) + '\n'
    print('cleaned')
    results = [] 
    for s in tqdm(nlp(final_text).sentences):
        results.append(' '.join([t.lemma for t in s.words if not t.lemma is None and t.lemma.lower() not in STOP_WORDS]))
    return results

In [17]:
se = Parallel(n_jobs=3)(delayed(run)(i) for i in tqdm(chunks(df['text'].values[:100000], 5000), total=df.shape[0] / 5000))



  0%|          | 0/3076.9334 [00:00<?, ?it/s]

  0%|          | 1/3076.9334 [00:00<17:39,  2.90it/s]

  0%|          | 6/3076.9334 [06:55<21:28:13, 25.17s/it]

  0%|          | 7/3076.9334 [07:15<20:03:03, 23.51s/it]

  0%|          | 8/3076.9334 [07:20<15:22:19, 18.03s/it]

  0%|          | 9/3076.9334 [08:07<22:44:16, 26.68s/it]

KeyboardInterrupt: 

In [12]:
len(se)

5

In [18]:
len(results)

3533

# combine files

In [2]:
data = []
for f in Path("/mnt/data2/ptf/final_zo").glob("*.json"):
    with open(f) as inp:
        for line in tqdm(inp):
            d = json.loads(line)
            if "comments" in d:
                for c in d["comments"]:
                    data.append({"url": d["url"], **c})

3328199it [00:38, 86966.48it/s]
3096it [00:00, 72439.03it/s]
1344582it [00:37, 35984.74it/s]


In [3]:
len(data)

15384667

In [4]:
df = pd.DataFrame(data)

# parse relative dates

In [5]:
def parse(x):
    # date when crawled 11th June 2019
    d = datetime.datetime(2019, 6, 11, 12, 0)
    idx = x.find('—')
    return dateparser.parse(x[idx:], languages=['de'], settings={'RELATIVE_BASE': d})

In [6]:
parsed = Parallel(n_jobs=4)(delayed(parse)(i) for i in tqdm(df['date'].values))

100%|██████████| 15384667/15384667 [3:10:19<00:00, 1347.22it/s]


In [8]:
df['date'] = parsed

In [10]:
df.to_pickle('parsed_data.pkl')

In [3]:
df = pd.read_pickle('parsed_data.pkl')

# group into chunks

In [4]:
df = df.sort_values(by='date', ascending=False)

In [5]:
df.shape

(15384667, 3)

In [6]:
df = df.drop_duplicates(subset=['text', 'date'])

In [7]:
df

,date,text,url
2145505,2019-06-11 11:59:50,"""Does a form of capitalism that generates incr...",https://www.zeit.de/wirtschaft/2019-05/kapital...
5176690,2019-06-11 11:59:49,Seehofer hat wiedermal einen Seehofer-Witz gem...,https://www.zeit.de/politik/deutschland/2019-0...
2368511,2019-06-11 11:59:48,"""In Deutschland wandern rund vier Tonnen Leben...",https://www.zeit.de/wirtschaft/2019-06/muellve...
6368951,2019-06-11 11:59:44,Vom Niveau nur knapp über dem Champions League...,https://www.zeit.de/sport/2019-06/uefa-nations...
2010560,2019-06-11 11:59:32,Mooomennt da kommt mir eine Idee...\nBäume mac...,https://www.zeit.de/wirtschaft/2019-06/klimasc...
1488465,2019-06-11 11:59:28,"Verständlich, dass sich Medien nach fast 20 Ja...",https://www.zeit.de/politik/deutschland/2019-0...
2368365,2019-06-11 11:59:11,So schlimm ist es nicht.\nAber leider orientie...,https://www.zeit.de/politik/deutschland/2019-0...
5765230,2019-06-11 11:59:07,Dieses Täuschungsmanöver..\nwäre bei keinem Ha...,https://www.zeit.de/sport/2019-06/sc-paderborn...
5367797,2019-06-11 11:59:05,"Der Kommentar, auf den Sie Bezug nehmen, wurde...",https://www.zeit.de/sport/2019-06/deutsche-fus...
2368643,2019-06-11 11:59:02,Mein Wattebausch kann sogar etwas aufnehmen un...,https://www.zeit.de/politik/ausland/2019-06/ge...


In [8]:
groups = pd.qcut(df['date'], 10)

In [9]:
groups

2145505           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5176690           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2368511           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
6368951           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2010560           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
1488465           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2368365           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5765230           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5367797           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2368643           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5367798           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2368611           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5709924           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2010174           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5862563           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
2145687           (2019-01-11 12:00:00, 2019-06-11 11:59:50]
5456641           (2019-

In [10]:
df['year'] = df['date'].apply(lambda x: x.year)

In [11]:
df['year'].value_counts()

2018    3257309
2017    2367352
2016    1680063
2019    1333176
2015    1064534
2014     751005
2013     494623
2012     481302
2011     427243
2010     226579
2009      79962
2008      22065
2006       3493
2007       1974
2005        102
Name: year, dtype: int64

In [12]:
df['group'] = df['year'].apply(lambda x: x if x > 2010 else 2010)

In [13]:
df['group'].value_counts()

2018    3257309
2017    2367352
2016    1680063
2019    1333176
2015    1064534
2014     751005
2013     494623
2012     481302
2011     427243
2010     334175
Name: group, dtype: int64

# clean, split into sentences

In [8]:
def get_sents(texts):
    tokenizer = Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
    sentence_splitter = SentenceSplitter(is_tuple=False)
    
    results = []
    for text in texts:
        text = clean(text, lang='de', lower=False)
        tokens = tokenizer.tokenize_paragraph(text)
        sentences = sentence_splitter.split(tokens)
        cleaned = [' '.join(s) for s in sentences]
        results.append(cleaned)
    return results

In [9]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [17]:
def combine(li):
    for l in li:
        for x in l:
            yield x

In [ ]:
results = Parallel(n_jobs=4)(delayed(get_sents)(row) for row in tqdm(list(chunks(df['text'], 10000))))

In [12]:
pickle.dump( results, open( "/mnt/data2/results_sentes.pkl", "wb" ) )

In [14]:
results = pickle.load( open( "/mnt/data2/results_sentes.pkl", "rb" ) )

In [26]:
df['sents'] = list(combine(results))

In [28]:
sents_data = []

In [ ]:
for _, row in tqdm(df[['group', 'sents']].iterrows(), total=df.shape[0]):
    for s in row['sents']:
        sents_data.append({'text': s, 'group': row['group']})

In [42]:
len(sents_data)

51973684

In [43]:
df_sents = pd.DataFrame(sents_data)

In [44]:
df_sents

,group,text
0,2019,""" Does a form of capitalism that generates inc..."
1,2019,"At present , democracy seems unable to find an..."
2,2019,""" Also das ist wirklich eine gute Zusammenfass..."
3,2019,"Fraglich ist , ob dieser Punkt schon erreicht ..."
4,2019,Seehofer hat wiedermal einen Seehofer-Witz gem...
5,2019,""" In Deutschland wandern rund vier Tonnen Lebe..."
6,2019,""" Na ja , wenn es nur 4 Tonnen sind , wo liegt..."
7,2019,Vom Niveau nur knapp über dem Champions League...
8,2019,Ist wohl die Strafe für eine absolut überragen...
9,2019,Mooomennt da kommt mir eine Idee ...


In [45]:
df_sents.to_pickle('/mnt/data2/results_sents.pkl')

In [2]:
df_sents = pd.read_pickle('/mnt/data2/results_sents.pkl')

# Lemmatize

In [3]:
df_sents = df_sents[df_sents['text'].str.len() > 10]

In [4]:
df_sents.shape

(50692789, 2)

In [5]:
res = list(lemmatize(df_sents['text'].values, n_jobs=4, chunk_size=100000))

100%|██████████| 507/507 [4:07:32<00:00, 18.54s/it]


In [8]:
df_sents['text'] = res

In [10]:
df_sents.to_pickle('/mnt/data2/results_sents_lemma.pkl')

In [21]:
def cl(x):
    return clean(x, fix_unicode=False, to_ascii=False, no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, lang='de')

In [ ]:
final = Parallel(n_jobs=4)(delayed(cl)(row) for row in tqdm(df_sents['text'].values))

In [23]:
len(final)

50692789

In [24]:
final[:10]

['does a form of capitalism that generates increasing inequality buen depends on mass consumption hav any other way out of its dilemma thanen to encourage households to take on unsustainable levels of deben',
 'at present democracy seems unable to find an answer to that questionen',
 'also das sein wirklich eine gut zusammenfassung der seltsam lage in der wir uns befinden und wahrscheinlich das einzig korrektiv die einkommensverteilung können nicht unendlich ungleich werden weil dann kein konsum mehr möglich sein',
 'fraglich sein ob dieser punkt schon erreichen sein und ich fürchten nicht denn das bedeuten die einkommensverteilung können noch weit ungleich werden',
 'seehofer haben wiedermal einen seehoferwitz machen über den außer seehofer niemand lachen können',
 'in deutschland wandern rund vier tonne lebensmittel laut bundesumweltamt jährlich direkt vom supermarktregal in die tonne',
 'na ja wenn es nur 0 tonne sein wo liegen das problem',
 'vom niveau nur knapp über dem champion 

In [25]:
df_sents['text'] = final

In [33]:
STOP_WORDS = set(
    """
á a ab aber ach acht achte achten achter achtes ag alle allein allem allen
aller allerdings alles allgemeinen als also am an andere anderen anderem andern
anders auch auf aus ausser außer ausserdem außerdem
bald bei beide beiden beim beispiel bekannt bereits besonders besser besten bin
bis bisher bist
da dabei dadurch dafür dagegen daher dahin dahinter damals damit danach daneben
dank dann daran darauf daraus darf darfst darin darüber darum darunter das
dasein daselbst dass daß dasselbe davon davor dazu dazwischen dein deine deinem
deiner dem dementsprechend demgegenüber demgemäss demgemäß demselben demzufolge
den denen denn denselben der deren derjenige derjenigen dermassen dermaßen
derselbe derselben des deshalb desselben dessen deswegen dich die diejenige
diejenigen dies diese dieselbe dieselben diesem diesen dieser dieses dir doch
dort drei drin dritte dritten dritter drittes du durch durchaus dürfen dürft
durfte durften
eben ebenso ehrlich eigen eigene eigenen eigener eigenes ein einander eine
einem einen einer eines einigeeinigen einiger einiges einmal einmaleins elf en
ende endlich entweder er erst erste ersten erster erstes es etwa etwas euch
früher fünf fünfte fünften fünfter fünftes für
gab ganz ganze ganzen ganzer ganzes gar gedurft gegen gegenüber gehabt gehen
geht gekannt gekonnt gemacht gemocht gemusst genug gerade gern gesagt geschweige
gewesen gewollt geworden gibt ging gleich gott gross groß grosse große grossen
großen grosser großer grosses großes gut gute guter gutes
habe haben habt hast hat hatte hätte hatten hätten heisst heißt her heute hier
hin hinter hoch
ich ihm ihn ihnen ihr ihre ihrem ihren ihrer ihres im immer in indem
infolgedessen ins irgend ist
ja jahr jahre jahren je jede jedem jeden jeder jedermann jedermanns jedoch
jemand jemandem jemanden jene jenem jenen jener jenes jetzt
kam kann kannst kaum kein keine keinem keinen keiner kleine kleinen kleiner
kleines kommen kommt können könnt konnte könnte konnten kurz
lang lange leicht leider lieber los
machen macht machte mag magst man manche manchem manchen mancher manches mehr
mein meine meinem meinen meiner meines mensch menschen mich mir mit mittel
mochte möchte mochten mögen möglich mögt morgen muss muß müssen musst müsst
musste mussten
na nach nachdem nahm natürlich neben nein neue neuen neun neunte neunten neunter
neuntes nicht nichts nie niemand niemandem niemanden noch nun nur
ob oben oder offen oft ohne
recht rechte rechten rechter rechtes richtig rund
sagt sagte sah satt schlecht schon sechs sechste sechsten sechster sechstes
sehr sei seid seien sein seine seinem seinen seiner seines seit seitdem selbst
selbst sich sie sieben siebente siebenten siebenter siebentes siebte siebten
siebter siebtes sind so solang solche solchem solchen solcher solches soll
sollen sollte sollten sondern sonst sowie später statt
tag tage tagen tat teil tel trotzdem tun
über überhaupt übrigens uhr um und uns unser unsere unserer unter
vergangene vergangenen viel viele vielem vielen vielleicht vier vierte vierten
vierter viertes vom von vor
wahr während währenddem währenddessen wann war wäre waren wart warum was wegen
weil weit weiter weitere weiteren weiteres welche welchem welchen welcher
welches wem wen wenig wenige weniger weniges wenigstens wenn wer werde werden
werdet wessen wie wieder will willst wir wird wirklich wirst wo wohl wollen
wollt wollte wollten worden wurde würde wurden würden
zehn zehnte zehnten zehnter zehntes zeit zu zuerst zugleich zum zunächst zur
zurück zusammen zwanzig zwar zwei zweite zweiten zweiter zweites zwischen
""".split()
)

In [34]:
def remove_stopwords(text):
    tokens = text.split()
    tokens = [t for t in tokens if not t in STOP_WORDS]
    return ' '.join(tokens)

In [ ]:
final = Parallel(n_jobs=4)(delayed(remove_stopwords)(row) for row in tqdm(df_sents['text'].values))

In [40]:
df_sents['text'] = final

In [43]:
df_sents.to_pickle('/mnt/data2/final_zo.pkl')

In [41]:
df_sents['text'] 

0           does form of capitalism that generates increas...
1           at present democracy seems unable to find answ...
2           zusammenfassung seltsam lage befinden wahrsche...
3           fraglich punkt erreichen fürchten bedeuten ein...
4             seehofer wiedermal seehoferwitz seehofer lachen
5           deutschland wandern tonne lebensmittel laut bu...
6                                      0 tonne liegen problem
7                         niveau knapp champion league finale
8                  strafe absolut überragend cl playoff phase
9                                              mooomennt idee
10                                        baum co0 sauerstoff
11          waldbesitzer äquivalent steuer sparen frage co...
12          verständlich medium fast 00 absolut uneitel pr...
13                                                        alt
14                        liegen sache ausführend person herz
15                                                    schlimm
16      

In [42]:
for year, group in df_sents.groupby('group'):
    print(year, group.shape)
    Path(f'/mnt/data2/ptf/groups/zo_{year}.txt').write_text('\n'.join(group['text'].values) + '\n')


2010 (1932455, 2)
2011 (2215604, 2)
2012 (2380169, 2)
2013 (2477431, 2)
2014 (3747399, 2)
2015 (4983872, 2)
2016 (6632652, 2)
2017 (8908685, 2)
2018 (11698714, 2)
2019 (5715808, 2)


In [2]:
df_sents = pd.read_pickle('/mnt/data2/final_zo.pkl')